In [1]:
#import dependencies and assign API key
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

eia_key = "3573bd97083abd7efae3399626d0ae59"

In [2]:
#identify series names and series ID for API data series call

#category API call - get series IDs for state level gasoline consumption
category_url = f"http://api.eia.gov/category/?api_key={eia_key}&category_id=404254"

#requests to get data from category url
data=requests.get(category_url).json()

#create empty lists for series_id and series_name
series_id=[]
series_name=[]

#iterate over the category and child series to extract 
for x in range(0, len(data['category']['childseries'])-1):
    series_id.append(data['category']['childseries'][x]['series_id'])
    series_name.append(data['category']['childseries'][x]['name'])

In [3]:
#place series ID and series names into a Dataframe
series_df = pd.DataFrame()
series_df["series_id"] = series_id
series_df["state summary"] = series_name
series_df.head()

,series_id,state summary
0,PET.C100000001.A,U.S. Total Gasoline All Sales/Deliveries by Pr...
1,PET.C100000001.M,U.S. Total Gasoline All Sales/Deliveries by Pr...
2,PET.C100010001.A,East Coast (PADD 1) Total Gasoline All Sales/D...
3,PET.C100010001.M,East Coast (PADD 1) Total Gasoline All Sales/D...
4,PET.C100011001.A,New England (PADD 1A) Total Gasoline All Sales...


In [4]:
#remove annual data from series_id's
remove_annual_df = series_df[~series_df["series_id"].str.contains(".A")]
remove_padd_df = remove_annual_df[~remove_annual_df["state summary"].str.contains("PADD")]
remove_padd_df = remove_padd_df.reset_index(drop=True)
remove_padd_df["state"] = remove_padd_df["state summary"].str.split('Total').str[0]
remove_padd_df.tail()

,series_id,state summary,state
46,PET.C100050041.M,Arizona Total Gasoline All Sales/Deliveries by...,Arizona
47,PET.C100050061.M,California Total Gasoline All Sales/Deliveries...,California
48,PET.C100050151.M,Hawaii Total Gasoline All Sales/Deliveries by ...,Hawaii
49,PET.C100050321.M,Nevada Total Gasoline All Sales/Deliveries by ...,Nevada
50,PET.C100050411.M,Oregon Total Gasoline All Sales/Deliveries by ...,Oregon


In [5]:
iter_ids = remove_padd_df["series_id"].values

In [6]:
new_df = pd.DataFrame()

data_url = f"http://api.eia.gov/series/?api_key={eia_key}&series_id=PET.C100000001.M"
series_data = requests.get(data_url).json()
st_cons = series_data['series'][0]['data']
new_df = pd.DataFrame(np.array(st_cons))
state_name = series_data['series'][0]['name']
new_df = new_df.rename(columns={0:"Month"})
new_df = new_df.rename(columns={1:state_name})
new_df.head()

,Month,"U.S. Total Gasoline All Sales/Deliveries by Prime Supplier, Monthly"
0,201812,363814.8
1,201811,372857.2
2,201810,378336.5
3,201809,371360.6
4,201808,390031.8


In [7]:
for id in iter_ids:
    
    data_url = f"http://api.eia.gov/series/?api_key={eia_key}&series_id={id}"
    series_data = requests.get(data_url).json()
    st_cons = series_data['series'][0]['data']
    df = pd.DataFrame(np.array(st_cons))
    state_name = series_data['series'][0]['name'].split("Total")
    df = df.rename(columns={0:"Month"})
    df = df.rename(columns={1:state_name[0]})
    new_df[f"{state_name[0]}"] = df[f"{state_name[0]}"]

In [8]:
new_df = new_df.drop(columns = ["U.S. Total Gasoline All Sales/Deliveries by Prime Supplier, Monthly"])
state_cons_df = new_df.set_index("Month")

In [9]:
state_cons_df = state_cons_df.T

In [13]:
state_cons_df = state_cons_df.loc[:, :'201601']
state_cons_df.head()

Month,201812,201811,201810,201809,201808,201807,201806,201805,201804,201803,...,201610,201609,201608,201607,201606,201605,201604,201603,201602,201601
U.S.,363814.8,372857.2,378336.5,371360.6,390031.8,383738.2,391639.9,386016,374734.3,374065.7,...,373717.8,375497.9,384718.7,385165.4,389669.5,373972.4,370840.6,366846.1,360546.8,342192.8
Connecticut,4226.8,4227.6,4165.6,4151.3,4359.6,4355.7,4479.9,4449.4,4196.5,4171,...,4223.5,4195.5,4290.5,4455.5,4408.3,4484.6,4606.9,4398.2,4337.7,4176.3
Maine,1877.8,1904.4,2075.5,2034.3,2266.8,2193.5,2164.7,2024.7,1796.7,1821.6,...,1986.5,2086.1,2252.3,2243.5,2091.3,1960.6,1857,1763.6,1785.3,1771.2
Massachusetts,6387,6459.8,6578.2,6475.5,6797.6,6674.5,6793.7,6694.4,6361.5,6194.4,...,7567.7,7615.9,7659.2,7810.1,7783.8,7011.9,7083.4,6871.9,6678.8,6563.6
New Hampshire,1553.5,1535.7,1612.4,1538.2,1656.1,1625,1626.5,1462.2,1419.1,1420.1,...,1541.5,1541.7,1641.1,1660.5,1595.2,1583.2,1489.8,1517.3,1572.9,1531.1


In [14]:
import pymysql
pymysql.install_as_MySQLdb()

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float

from sqlalchemy import create_engine

dbuser = 'root'
dbpassword = 'ro-Cha7t'
dbhost = 'localhost'
dbport = '3306'
dbname= 'ETL_db'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")
Base.metadata.create_all(engine)

In [15]:
# state_cons_df
state_cons_df.to_sql('cons', engine, if_exists="append")